# DSAI HW2  
## Chien, Hsin Yen
### RE6071088, Institute of Data Science  

In [43]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
import numpy as np
from six.moves import range

# Parameters Config

In [44]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [45]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [46]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [47]:
ctable = CharacterTable(chars)

In [48]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [49]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [50]:
print(questions[:5], expected[:5])

['70+7   ', '34+89  ', '7+14   ', '250+31 ', '798+612'] ['77  ', '123 ', '21  ', '281 ', '1410']


# Processing

In [51]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [52]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False False  True
   False]
  [False False False False False  True False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False False False False False False False False False False
    True]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False  True
   False]
  [False False False False  True False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False False False  True False False False
   False]
  [ True False False False False False Fal

# Build Model

In [30]:
print('Build model...')

# Initialising the RNN
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for i in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_31 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_32 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [31]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 343us/step - loss: 2.0227 - acc: 0.3002 - val_loss: 1.9424 - val_acc: 0.3135
Q 940+226 T 1166 ☒ 111 
Q 699+71  T 770  ☒ 114 
Q 867+249 T 1116 ☒ 1110
Q 460+844 T 1304 ☒ 111 
Q 110+754 T 864  ☒ 115 
Q 0+94    T 94   ☒ 114 
Q 87+170  T 257  ☒ 114 
Q 654+8   T 662  ☒ 114 
Q 980+62  T 1042 ☒ 114 
Q 556+16  T 572  ☒ 114 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 232us/step - loss: 1.9033 - acc: 0.3209 - val_loss: 1.8613 - val_acc: 0.3251
Q 130+2   T 132  ☒ 13  
Q 502+306 T 808  ☒ 130 
Q 850+0   T 850  ☒ 147 
Q 64+201  T 265  ☒ 137 
Q 2+297   T 299  ☒ 147 
Q 2+512   T 514  ☒ 13  
Q 66+990  T 1056 ☒ 110 
Q 851+84  T 935  ☒ 107 
Q 187+66  T 253  ☒ 107 
Q 3+441   T 444  ☒ 13  

-------------------

18000/18000 [==============================] - 5s 258us/step - loss: 1.2730 - acc: 0.5241 - val_loss: 1.3000 - val_acc: 0.5031
Q 66+573  T 639  ☒ 633 
Q 2+64    T 66   ☑ 66  
Q 806+76  T 882  ☒ 773 
Q 653+310 T 963  ☒ 966 
Q 163+161 T 324  ☒ 278 
Q 657+66  T 723  ☒ 722 
Q 95+188  T 283  ☒ 267 
Q 721+71  T 792  ☒ 783 
Q 389+395 T 784  ☒ 877 
Q 3+700   T 703  ☒ 70  

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 273us/step - loss: 1.2386 - acc: 0.5382 - val_loss: 1.2404 - val_acc: 0.5304
Q 97+676  T 773  ☒ 754 
Q 51+215  T 266  ☒ 157 
Q 7+329   T 336  ☒ 346 
Q 737+21  T 758  ☒ 745 
Q 1+688   T 689  ☒ 157 
Q 494+21  T 515  ☒ 449 
Q 50+276  T 326  ☒ 235 
Q 622+32  T 654  ☒ 644 
Q 81+170  T 251  ☒ 287 
Q 995+6   T 1001 ☒ 194 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 238us/step - loss: 0.2730 - acc: 0.9168 - val_loss: 0.3280 - val_acc: 0.8871
Q 4+853   T 857  ☑ 857 
Q 31+112  T 143  ☒ 133 
Q 351+655 T 1006 ☑ 1006
Q 1+573   T 574  ☑ 574 
Q 972+748 T 1720 ☒ 1710
Q 997+815 T 1812 ☒ 1710
Q 8+694   T 702  ☒ 701 
Q 959+65  T 1024 ☑ 1024
Q 789+889 T 1678 ☑ 1678
Q 61+304  T 365  ☑ 365 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 239us/step - loss: 0.2578 - acc: 0.9224 - val_loss: 0.2846 - val_acc: 0.9042
Q 359+563 T 922  ☑ 922 
Q 559+461 T 1020 ☑ 1020
Q 50+133  T 183  ☑ 183 
Q 569+728 T 1297 ☑ 1297
Q 14+826  T 840  ☑ 840 
Q 98+760  T 858  ☑ 858 
Q 843+90  T 933  ☑ 933 
Q 7+805   T 812  ☑ 812 
Q 957+21  T 978  ☑ 978 
Q 73+22   T 95   ☒ 96  

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 237us/step - loss: 0.0757 - acc: 0.9821 - val_loss: 0.3080 - val_acc: 0.8938
Q 906+684 T 1590 ☒ 1591
Q 86+389  T 475  ☒ 476 
Q 90+76   T 166  ☒ 167 
Q 237+684 T 921  ☑ 921 
Q 885+62  T 947  ☒ 948 
Q 721+74  T 795  ☑ 795 
Q 721+81  T 802  ☒ 702 
Q 15+839  T 854  ☑ 854 
Q 39+978  T 1017 ☑ 1017
Q 893+58  T 951  ☒ 952 

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 250us/step - loss: 0.0709 - acc: 0.9833 - val_loss: 0.1056 - val_acc: 0.9691
Q 7+802   T 809  ☑ 809 
Q 31+958  T 989  ☑ 989 
Q 45+193  T 238  ☑ 238 
Q 66+75   T 141  ☑ 141 
Q 384+806 T 1190 ☑ 1190
Q 46+85   T 131  ☑ 131 
Q 0+716   T 716  ☑ 716 
Q 882+58  T 940  ☑ 940 
Q 647+48  T 695  ☑ 695 
Q 92+813  T 905  ☑ 905 

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 249us/step - loss: 0.0327 - acc: 0.9923 - val_loss: 0.0808 - val_acc: 0.9741
Q 35+985  T 1020 ☑ 1020
Q 986+9   T 995  ☑ 995 
Q 44+27   T 71   ☑ 71  
Q 190+4   T 194  ☑ 194 
Q 91+43   T 134  ☑ 134 
Q 815+96  T 911  ☑ 911 
Q 227+241 T 468  ☑ 468 
Q 656+95  T 751  ☑ 751 
Q 86+281  T 367  ☑ 367 
Q 307+911 T 1218 ☑ 1218

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 262us/step - loss: 0.0389 - acc: 0.9914 - val_loss: 0.0908 - val_acc: 0.9704
Q 935+829 T 1764 ☑ 1764
Q 309+759 T 1068 ☑ 1068
Q 835+628 T 1463 ☑ 1463
Q 7+802   T 809  ☑ 809 
Q 642+192 T 834  ☒ 824 
Q 82+611  T 693  ☑ 693 
Q 14+22   T 36   ☑ 36  
Q 91+56   T 147  ☑ 147 
Q 354+91  T 445  ☑ 445 
Q 834+214 T 1048 ☑ 1048


# Testing 1 (by test data)

In [54]:
print("MSG : Prediction")
preds = model.predict_classes(test_x)
for i in range(10):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)

MSG : Prediction
Q 415+13  T 428  ☑ 428 
Q 35+76   T 111  ☑ 111 
Q 157+28  T 185  ☑ 185 
Q 74+606  T 680  ☑ 680 
Q 122+67  T 189  ☑ 189 
Q 707+47  T 754  ☑ 754 
Q 60+257  T 317  ☑ 317 
Q 102+160 T 262  ☒ 271 
Q 417+308 T 725  ☑ 725 
Q 404+359 T 763  ☑ 763 


# Testing 2 (by new question)
Q: 555+175, 860+7  , 340+29

In [58]:
newQ = ['555+175', '860+7  ', '340+29 ']
newA = ['730 ', '867 ', '369 ']
print('Vectorization...')
x = np.zeros((len(newQ), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(newA), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(newQ):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(newA):
    y[i] = ctable.encode(sentence, DIGITS + 1)
    
print("MSG : Prediction")
for i in range(len(newQ)):
    preds = model.predict_classes(x)
    q = ctable.decode(x[i])
    correct = ctable.decode(y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)

Vectorization...
MSG : Prediction
Q 555+175 T 730  ☑ 730 
Q 860+7   T 867  ☑ 867 
Q 340+29  T 369  ☑ 369 
